In [1]:
# Import relevant modules
import numpy as np
import pandas as pd
import datetime as dt
from queue import PriorityQueue as PQ
from geopy import distance
from time import time as t
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'

In [2]:
# Load Stadium information (from Chat GPT) and get team abbreviations 
stadiums = pd.read_csv('NHL Stadium Locations.csv')
teamAbbrev = list(stadiums['teamAbbrev'])
stadiums = stadiums.set_index('teamAbbrev')
stadiums.head()

,Team,Stadium,Longitude,Latitude
teamAbbrev,,,,
ANA,Anaheim Ducks,Honda Center,-117.8763,33.8075
ARI,Arizona Coyotes,Gila River Arena,-112.1886,33.5317
BOS,Boston Bruins,TD Garden,-71.0621,42.3662
BUF,Buffalo Sabres,KeyBank Center,-78.8756,42.8750
CGY,Calgary Flames,Scotiabank Saddledome,-114.0555,51.0374


In [3]:
# Load all games from the 2023-24 NHL Season (from NHL API... i love you NHL for setting up an API)
allGames = pd.read_csv('2023-24 NHL Scheduled Games.csv')
# Pandas doesn't load dictionaries in csv's so eval the strings
allGames['awayTeam'] = [eval(i) for i in allGames['awayTeam']]
allGames['homeTeam'] = [eval(i) for i in allGames['homeTeam']]
allGames.head()

,homeTeam,awayTeam
0,"{'id': 54, 'placeName': {'default': 'Vegas'}, ...","{'id': 24, 'placeName': {'default': 'Anaheim'}..."
1,"{'id': 24, 'placeName': {'default': 'Anaheim'}...","{'id': 12, 'placeName': {'default': 'Carolina'..."
2,"{'id': 24, 'placeName': {'default': 'Anaheim'}...","{'id': 25, 'placeName': {'default': 'Dallas'},..."
3,"{'id': 53, 'placeName': {'default': 'Arizona'}...","{'id': 24, 'placeName': {'default': 'Anaheim'}..."
4,"{'id': 24, 'placeName': {'default': 'Anaheim'}...","{'id': 6, 'placeName': {'default': 'Boston'}, ..."


In [4]:
# Used for translating team abbreviation to their NHL id
teamId = {row['abbrev']:row['id'] for row in allGames['awayTeam']}

In [5]:
teamToId = lambda team: teamId[team]

In [6]:
def dist(team1, team2):
    if (team1 is None) or (team2 is None):
        return 0
    loc1 = stadiums.loc[team1]
    loc2 = stadiums.loc[team2]
    coord1 = (loc1['Latitude'], loc1['Longitude'])
    coord2 = (loc2['Latitude'], loc2['Longitude'])
    return distance.distance(coord1, coord2).miles

In [7]:
def randomTeam(*notThis):
    return np.random.choice([i for i in teamAbbrev if i not in notThis])

In [8]:
ref = stadiums.reset_index()

In [9]:
data = []
for loc1 in range(32):
    for loc2 in range(loc1+1, 32):
        location1 = ref.iloc[loc1]['teamAbbrev']
        location2 = ref.iloc[loc2]['teamAbbrev']
        data += [{'From': location1,
                 'To': location2,
                 'Distance': dist(location1, location2)}]

In [10]:
ref = pd.DataFrame(data)

In [11]:
ref.sort_values('Distance', ascending = False).head()

,From,To,Distance
321,FLA,VAN,2779.811586
317,FLA,SEA,2710.576434
81,BOS,SJS,2685.340019
71,BOS,LAK,2597.870474
482,TBL,VAN,2596.936061


In [12]:
# Define classes needed to run the search
class Game:
    def __init__(self, home, away):
        self.home = home
        self.away = away
        self.teams = (home, away)
        self.marked = False
        self.hash = 100*teamToId(away) + teamToId(home)
    
    def mark(self, date):
        self.date = date
        self.hash += 100000*(date.day+(100*date.month)+(10000*date.year)) + 10000
        self.marked = True
        
    def homeT(self, team):
        return self.home == team
    
    def awayT(self, team):
        return self.away == team
        
    def involves(self, team):
        return (self.home == team) or (self.away == team)
    
    def __repr__(self):
        return f'{self.away} @ {self.home}'
    
    def __eq__(self, other):
        if not isinstance(other, Game):
            return False
        return self.hash == other.hash
    
    def __lt__(self, other):
        if not isinstance(other, Game):
            return False
        return self.hash < other.hash
    
    def __gt__(self, other):
        if not isinstance(other, Game):
            return False
        return self.hash > other.hash
    
    def copy(self):
        newGame = Game(self.home, self.away)
        if self.marked:
            newGame.mark(self.date)
        return newGame
    
    def opponent(self, team):
        if team == self.home:
            return self.away
        if team == self.away:
            return self.home
        return None
    
    def __hash__(self):
        return self.hash

    
class Team:
    def __init__(self, name):
        self.name = name
        self.lastGameDate = dt.date.min
        self.lastGameLocation = name
        self.homeStand = 0
        self.roadTrip = 0
        self.gamesRemaining = 82
        self.gameStreak = 0
        
    def __repr__(self):
        return self.name
    
    def copy(self):
        newTeam = Team(self.name)
        newTeam.lastGameDate = self.lastGameDate
        newTeam.lastGameLocation = self.lastGameLocation
        newTeam.homeStand = self.homeStand
        newTeam.roadTrip = self.roadTrip
        newTeam.gamesRemaining = self.gamesRemaining
        newTeam.gameStreak = self.gameStreak
        return newTeam
    
    def schedule(self, game):
        assert self.lastGameDate <= game.date
        distance = dist(self.lastGameLocation, game.home)
        daysBetween = 3 - min([(game.date - self.lastGameDate).days-1,3])
        cost = distance ** (1+(.1*daysBetween))
        self.lastGameDate = game.date
        self.lastGameLocation = game.home
        self.gamesRemaining -= 1
        return cost
    
    def scheduleHome(self, game):
        self.homeStand += 1
        self.roadTrip = 0
        return self.schedule(game)
        
    def scheduleAway(self, game):
        self.roadTrip += 1
        self.homeStand = 0
        return self.schedule(game)
    
    def advanceDay(self, date):
        if self.lastGameDate == date:
            self.gameStreak += 1
            return 1
        self.gameStreak = 0
        return 0
    
    def valid(self, game, scheduledGames, date):
        '''Check which potential games we can schedule that day based on the following criteria:
        1. The team can't play twice in a day
        2. The team can't play the same team three times within a week
        3. Team cannot play a back-to-back-to-back
        4. Teams cannot play more than three games in a seven day span
        5. Team cannot have a home stand for more than five games
        6. Team cannot have a road trip for more than five games
        7. Team cannot play the same matchup in a back-to-back'''
        # Can't play twice in a day
        #print(date)
        if date == self.lastGameDate:
            #print('INVALID: Already played today')
            return False
        opponent = game.opponent(self.name)
        scheduledGames = [x for x in scheduledGames if x.involves(self.name)]
        withinWeek = [scheduledGame.opponent(self.name) == opponent for scheduledGame in scheduledGames if (date - scheduledGame.date).days <=7]
        # Don't want the same matchup three times in a week
        if sum(withinWeek) >= 2:
            #print('INVALID: Same matchup 3 times in a week')
            return False
        # Don't want three games in a row
        withinDays = [scheduledGame for scheduledGame in scheduledGames if (date - scheduledGame.date).days <= 2]
        if len(withinDays) >= 2:
            #print('INVALID: 3 Games in a Row')
            return False
        # Don't want more than three games in a five day span
        withinDays = [scheduledGame for scheduledGame in scheduledGames if (date - scheduledGame.date).days <= 7]
        if len(withinDays) >= 3:
            #print('INVALID: Playing more than 3 games in a 7 day span')
            return False
        # Don't want a home stand extending past 5 games
        if game.homeT(self.name) and (self.homeStand >= 5):
            #print('INVALID: Homestand over 5 games')
            return False
        # Don't want a road trip extending past 5 games
        if game.awayT(self.name) and (self.roadTrip >= 5):
            #print('INVALID: Road Trip over 5 games')
            return False
        # If no games scheduled forego the back-to-back check
        if len(scheduledGames) == 0:
            return True
        # Make sure there isn't the same matchup back-to-back
        recentGame = max(scheduledGames)
        return not ((recentGame.home == game.home) and (recentGame.away == game.away))


class Schedule:
    def __init__(self, toSchedule, scheduled, sznEnd, date, teamObj, mandatory_offs = []):
        self.scheduled_games = scheduled
        self.unscheduled_games = toSchedule
        self.date = date
        self.sznEnd = sznEnd
        self.teams = teamObj
        self.cost = 0
        self.mandatory_offdays = mandatory_offs
    
    def copy(self):
        teamCopy = {i: self.teams[i].copy() for i in self.teams.keys()}
        unsched = [i.copy() for i in self.unscheduled_games]
        sched = [i.copy() for i in self.scheduled_games]
        newSched = Schedule(unsched, sched, self.sznEnd, self.date, teamCopy, self.mandatory_offdays)
        newSched.cost = self.cost
        return newSched
        
    def markGame(self, game):
        '''Returns the cost of marking'''
        for gameToSchedule in self.unscheduled_games:
            if gameToSchedule == game:
                gameToSchedule.mark(self.date)
                break
        self.scheduled_games += [gameToSchedule]
        self.scheduled_games = sorted(self.scheduled_games)
        cost1 = self.teams[game.home].scheduleHome(gameToSchedule)
        cost2 = self.teams[game.away].scheduleAway(gameToSchedule)
        self.unscheduled_games = sorted([i for i in self.unscheduled_games if not i.marked])
        return cost1 + cost2
    
    def heuristic(self):
        locations = {team:{self.teams[team].lastGameLocation} for team in teamAbbrev}
        for game in self.unscheduled_games:
            for team in game.teams:
                locations[team] = locations[team] | {game.home}
        h = 0
        for game in self.unscheduled_games:
            h += min([dist(i, game.home) for i in locations[game.home]])
            h += min([dist(i, game.home) for i in locations[game.away]])
        return h
        
    def advanceDay(self):
        totalGames = 0
        for team in self.teams.values():
            totalGames += team.advanceDay(self.date)
        self.date += dt.timedelta(days = 1)
        return totalGames
        
    def getSuccessor(self, action):
        '''Returns successor state and the cost of it'''
        successor = self.copy()
        cost = 0
        if action == 'ADVANCE DAY':
            totalGames = successor.advanceDay()
            cost = 1500**(1-(.1*totalGames))
        elif action == 'ADVANCE DAY WITH NO PENALTY':
            successor.advanceDay()
        else:
            cost += successor.markGame(action)
        successor.cost += cost
        return successor
        
    def getLegalActions(self):
        # During Christmas break, schedule no games
        if self.date in self.mandatory_offdays:
            return ['ADVANCE DAY WITH NO PENALTY']
        # If we skip to the point where a 3 in a row needs to be played, terminate the season
        daysLeft = (self.sznEnd - self.date).days
        for team in self.teams.values():
            if (daysLeft-4) < (7*(team.gamesRemaining-3)/3): 
                return []
        bothValid = lambda x: self.teams[x.home].valid(x, self.scheduled_games, self.date) and self.teams[x.away].valid(x, self.scheduled_games, self.date)
        validGames = [x for x in set(self.unscheduled_games) if bothValid(x)]
        validGames += ['ADVANCE DAY']
        return validGames
    
    def isGoalState(self):
        #print(self.unscheduled_games)
        return len(self.unscheduled_games) == 0
    
    def __eq__(self, other):
        if not isinstance(other, Schedule):
            return False
        if len(self.unscheduled_games) != len(other.unscheduled_games):
            return False
        if self.date != other.date:
            return False
        if self.scheduled_games != other.scheduled_games:
            return False
        return True
    
    def __lt__(self,other):
        if not isinstance(other, Schedule):
            return False
        return len(self.unscheduled_games) == len(other.unscheduled_games)
    
    def __str__(self):
        string = ''
        for game in self.scheduled_games:
            string += f'{game.date}: {game}\n'
        return string
    
class bookmark:
    # More efficient searching for visited states
    def __init__(self):
        self.storage = {}
    
    def visit(self, sched):
        numScheduled = len(sched.scheduled_games)
        if numScheduled not in self.storage:
            self.storage[numScheduled] = []
        visited = sched in self.storage[numScheduled]
        if not visited:
            self.storage[numScheduled] = self.storage[numScheduled] + [sched]
        return visited

In [13]:
gameToObj = lambda x: Game(x['homeTeam']['abbrev'], x['awayTeam']['abbrev'])

In [14]:
def runSearch(gamesToSchedule, startDate = dt.date(2023,10,10), endDate = dt.date(2024,4,18), allowedDays = None):
    '''Input: 
    gamesToSchedule- list of Game objects to schedule
    startDate- date object of the startDate (default = 10/10/2023)
    end date- date object of the endDate (default = 4/18/2024)
    allowedDays- allowedDays (default = None), overrides endDate
    Output: Schedule object with optimal ordering'''
    if allowedDays is not None:
        endDate = startDate + dt.timedelta(days = allowedDays - 1)
    # Initialize Priority Queue and Team objects
    order = PQ()
    def makeTeam(abbrev):
        tm = Team(abbrev)
        tm.gamesRemaining = sum([game.involves(abbrev) for game in gamesToSchedule])
        return tm
    teams = {team : makeTeam(team) for team in teamAbbrev}
    #Add initial state
    initial_state = Schedule(gamesToSchedule, [], endDate, startDate, teams)
    order.put((0,initial_state))
    visited = bookmark()
    while not order.empty():
        state = order.get()[1]
        if visited.visit(state):
            continue
        #print(f'Scheduled: {state.scheduled_games}')
        #print(f"Game Streak: {state.teams['CAR'].roadTrip}")
        if state.isGoalState():
            print('Found Solution')
            return state
        legalActions = state.getLegalActions()
        for action in legalActions:
            successor= state.getSuccessor(action)
            p = successor.cost + successor.heuristic()
            #print(action, p)
            order.put((p, successor))
    return 'No Solution'

# Test Cases

In [15]:
# Three games in 2 days should be None
runSearch([Game('CAR', 'PHI'), Game('CAR', 'NYR'), Game('NYI', 'CAR')], allowedDays = 2)

'No Solution'

In [16]:
# Three games in 3 days should also be None
print(runSearch([Game('CAR', 'PHI'), Game('CAR', 'NYR'), Game('NYI', 'CAR')], allowedDays = 3))

No Solution


In [17]:
# Three games in 4 days should work
sched = runSearch([Game('CAR', 'PHI'), Game('CAR', 'NYR'), Game('NYI', 'CAR')], allowedDays = 4)
print(sched)

No Solution


In [18]:
# Four games in 7 days shouldn't work
runSearch([Game('CAR', 'PHI'), Game('CAR', 'NYR'), Game('NYI', 'CAR'), Game('ARI', 'CAR')], allowedDays = 7)

'No Solution'

In [19]:
# More than 5 home games in a row shouldn't work
runSearch([Game('CAR',randomTeam('CAR')) for i in range(6)], allowedDays = 14)

'No Solution'

In [20]:
# More than 5 away games in a row shouldn't work
runSearch([Game(randomTeam('CAR'), 'CAR') for i in range(6)], allowedDays = 14)

'No Solution'

In [21]:
# 6 Games in 14 days should work
tstGames = [Game(randomTeam('CAR'), 'CAR') for i in range(3)] + [Game('CAR', randomTeam('CAR')) for i in range(3)]
print(tstGames)
sched = runSearch(tstGames, allowedDays = 14)
for i in sched.scheduled_games:
    print(f'{i}:{i.date}')

[CAR @ CHI, CAR @ TBL, CAR @ SJS, OTT @ CAR, PHI @ CAR, ARI @ CAR]
Found Solution
OTT @ CAR:2023-10-10
ARI @ CAR:2023-10-11
CAR @ TBL:2023-10-15
PHI @ CAR:2023-10-18
CAR @ CHI:2023-10-20
CAR @ SJS:2023-10-23


In [22]:
# No back to back same matchups
runSearch([Game('CAR', 'NYR'), Game('CAR', 'NYR')], allowedDays = 3)

'No Solution'

In [ ]:
# 7 games, 6 road. Make sure the road trip is split
tstGames = [Game(randomTeam('CAR'), 'CAR') for i in range(6)] + [Game('CAR', randomTeam('CAR')) for i in range(1)]
print(tstGames)
sched = runSearch(tstGames, allowedDays = 28)
print(sched)

[CAR @ MIN, CAR @ NYR, CAR @ CGY, CAR @ MIN, CAR @ MTL, CAR @ EDM, PIT @ CAR]


In [ ]:
# 7 games, 6 home. Make sure the home stand is split
tstGames = [Game(randomTeam('CAR'), 'CAR') for i in range(1)] + [Game('CAR', randomTeam('CAR')) for i in range(6)]
print(tstGames)
sched = runSearch(tstGames, allowedDays = 28)
print(sched)

# Time Complexity

In [ ]:
def timeRandom(n):
    '''Time a search for n games, assume enough days'''
    randomGames = list(allGames.sample(n = n).apply(gameToObj, axis = 1))
    t1 = t()
    sol = runSearch(randomGames)
    return sol, t() - t1

In [ ]:
x, y, solutions = [], [], []

In [ ]:
numSims = 0
for n in range(1,6):
    for rep in range(30):
        x += [n]
        sol = timeRandom(n)
        numSims += 1
        print(f'{numSims}/150')
        y += [sol[1]]
        solutions += [sol[0]]

In [ ]:
timeData = pd.DataFrame({'n': x, 'n^2':[i**2 for i in x], 'Time (sec)': y})

In [ ]:
from sklearn.linear_model import LinearRegression as linreg

In [ ]:
model = linreg(fit_intercept = False)

In [ ]:
model.fit(X = timeData[['n', 'n^2']], y = timeData['Time (sec)'])

In [ ]:
sns.regplot(x,y, order = 2)
# xs = np.linspace(1,7,1000)
# plt.plot(xs, model.predict([[i,i**2]for i in xs]))
plt.xlabel('# of games searched')
plt.ylabel('Time to find Solution (seconds)')
plt.title('The Search Algorithm Runs on Exponential Time')

In [ ]:
fullPred = model.predict([[1312, 1312**2]])[0]

In [ ]:
# Around 339 days to search for the optimal NHL schedule
print(f'It will likely take around {round(fullPred/60/60/24)} days to run this for the full 2022 schedule')